In [3]:
import os
import sys

project_path = "/home/nlp/achimoa/workspace/hebrew_text_retrieval"
src_path = os.path.join(project_path, "src")

os.chdir(project_path)
if src_path not in sys.path:
    sys.path.append(src_path)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Seed tbe eval dataset with HeQ evaluation data.

In [6]:
from datasets import load_dataset

heq_dataset = load_dataset('pig4431/HeQ_v1', split='test', trust_remote_code=True, num_proc=24)
heq_dataset

Dataset({
    features: ['ID', 'Title', 'Source', 'Context', 'Question', 'Answers', 'Is_Impossible', 'WH_Question', 'Question_Quality'],
    num_rows: 1504
})

In [20]:
data_files = {
    "test": "data/retrieval/documents/test.jsonl"
}

documents_dataset = load_dataset("json", data_files=data_files)
documents_dataset

DatasetDict({
    test: Dataset({
        features: ['text', '_source', '_row_number', '_file'],
        num_rows: 332273
    })
})

In [24]:
import re
from datasets import Dataset
from tqdm import tqdm

data = []
for example in tqdm(documents_dataset["test"]):
    if example["_source"] != "heq":
        for paragraph in example["text"].split("\n"):
            paragraph = re.sub(r'\[\d+\]', '', paragraph).strip()
            if len(paragraph) > 100:
                data.append({
                    **example,
                    "text": paragraph,
                })
    else:
        data.append(example)

    
documents_dataset2 = Dataset.from_list(data)

documents_dataset2.to_json("data/retrieval/documents/test2.jsonl", orient="records", lines=True, force_ascii=False)

documents_dataset2

100%|██████████| 332273/332273 [00:27<00:00, 11998.86it/s]


Creating json from Arrow format:   0%|          | 0/3187 [00:00<?, ?ba/s]

Dataset({
    features: ['text', '_source', '_row_number', '_file'],
    num_rows: 3186348
})

In [25]:
data_files = {
    "test": "data/retrieval/documents/test2.jsonl"
}

documents_dataset = load_dataset("json", data_files=data_files)
documents_dataset

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['text', '_source', '_row_number', '_file'],
        num_rows: 3186348
    })
})

In [27]:
from collections import Counter

# Extract the _source column
source_values = documents_dataset["test"]["_source"]

# Count frequency of each unique value
source_distribution = Counter(source_values)

# Print the distribution sorted by frequency
for value, count in source_distribution.most_common():
    print(f"{value}: {count}")


AllOfNewHebrewWikipediaWithArticles-Oct29-2023.forgpt.jsonl: 275000
GeektimeCorpus-Combined-Deduped.forgpt.jsonl: 32600
AllOfNewHebrewWikipediaWithArticles-Oct29.forgpt.jsonl: 24564
heq: 109


In [28]:
documents_dataset["test"]

Dataset({
    features: ['text', '_source', '_file', '_row_number'],
    num_rows: 332273
})

In [ ]:
from datasets import load_dataset
import re

# queries
data_files = {
    "documents": {
        "test": "data/retrieval/documents/test.jsonl"
    }
}

documents_dataset = load_dataset("json", data_files=data_files["documents"], split="test")

print("Wiki")
counter = 0
for example in documents_dataset:
    if "Wikipedia" in example["_source"]:
        for paragraph in example['text'].split('\n'):
            paragraph = re.sub(r'(?:\[\d+\])+', '', paragraph)
            if len(paragraph) > 100:
                print(paragraph)
                print(len(paragraph))
        counter += 1
        if counter >= 10:
            break

print("Geektime")
counter = 0
for example in documents_dataset:
    if "Geektime" in example["_source"]:
        for paragraph in example['text'].split('\n'):
            if len(paragraph) > 100:
                print(paragraph)
                print(len(paragraph))
        counter += 1
        if counter >= 10:
            break

print("HeQ")
counter = 0
for example in documents_dataset:
    if "heq" in example["_source"]:
        print(example['text'])
        print(len(example['text']))
        counter += 1
        if counter >= 10:
            break

Wiki
"דלק" חברת הדלק הישראלית בע"מ, הידועה יותר בשם דלק, היא אחת מארבע חברות הדלק הגדולות בישראל. בבעלותה מאות תחנות דלק ברחבי הארץ, בחלקן יש גם חנויות נוחות תחת המותג "מנטה" ומתחמים קמעונאיים. בבעלות החברה מספר חברות-בנות בתחומי האנרגיה והלוגיסטיקה, המספקות שירותי שינוע והובלה של דלק, מערכות לוגיסטיות, ייצור שמנים וחומרי איטום, ושירותי ניפוק לאוניות בנמלים בישראל ובעולם.
369
עם הקמת מדינת ישראל שתי חברות שיווק הדלק שפעלו במדינה (חברות הבת המקומיות של תאגידי הנפט רויאל דאטש של ו"סוקוני ואקום") היו בבעלות זרה. גם בתי זיקוק לנפט בחיפה נשארו בבעלות זרה, אך פעילות צינור הנפט הגולמי מעיראק הופסקה באפריל 1948. חברות שיווק הדלק המשיכו לפעול עם המלאים שנותרו, אך לאחר מכן הושבתו במשך שנתיים. ייבוא הנפט היה ההוצאה הגדולה ביותר של המדינה במטבע זר, ונשלט כולו על ידי חברות זרות. לפיכך החליטה הממשלה להקים חברה בבעלות ישראלית, ולחסוך מטבע זר, בדמות הרווחים שעברו קודם לקופות החברות הבינלאומיות מהעסקים של יבוא הנפט, זיקוקו ושיווקו. החברה הוקמה על ידי ממשלת ישראל בדצמבר 1951 בשם "חברת הדלק הישראלית בע"מ

In [29]:
seen = set()
duplicates = []

for example in documents_dataset["test"]:
    key = example["text"]
    if key in seen:
        duplicates.append(key)
    else:
        seen.add(key)

print(f"Total examples: {len(documents_dataset["test"])}")
print(f"Duplicate pairs: {len(duplicates)}")

Total examples: 332273
Duplicate pairs: 0


In [ ]:
heq_dataset = load_dataset("pig4431/HeQ_v1", split="test")
heq_dataset

Dataset({
    features: ['ID', 'Title', 'Source', 'Context', 'Question', 'Answers', 'Is_Impossible', 'WH_Question', 'Question_Quality'],
    num_rows: 1504
})

In [14]:
seen = set()
duplicates = []

for example in heq_dataset:
    key = (example["Question"], example["Context"])
    if key in seen:
        duplicates.append(key)
    else:
        seen.add(key)

print(f"Total examples: {len(heq_dataset)}")
print(f"Duplicate (Question, Context) pairs: {len(duplicates)}")

Total examples: 1504
Duplicate (Question, Context) pairs: 1


In [15]:
from collections import defaultdict

# Track records by (Question, Context)
record_map = defaultdict(list)

for idx, example in enumerate(heq_dataset):
    key = (example["Question"], example["Context"])
    record_map[key].append((idx, example))

# Extract duplicates
duplicates = {key: entries for key, entries in record_map.items() if len(entries) > 1}

print(f"Found {len(duplicates)} duplicated (Question, Context) pairs.\n")

# Show examples of duplicated records
for key, entries in list(duplicates.items())[:10]:  # Limit to first 10 duplicates
    print(f"\n❗ Duplicate (Question, Context) pair:")
    print(f"Question: {key[0]}")
    print(f"Context: {key[1][:200]}...")  # Truncate for readability
    print("Occurrences:")
    for idx, record in entries:
        print(f" - Record index: {idx}, ID: {record['ID']}")


Found 1 duplicated (Question, Context) pairs.


❗ Duplicate (Question, Context) pair:
Question: בכמה כסף נמכרה צ׳ק פוינט?
Context: זוהי הרכישה ה-16 של צ'ק פוינט מאז הקמתה, והרכישה הישראלית ה-6 שלה ב-3 השנים האחרונות, לה קדמו הרכישות של Protego ב-40 מיליון דולר, Cymplify שנרכש ב-6 מיליון דולר בלי מוצר, ForceNock שנרכש תוך שנתיים מ...
Occurrences:
 - Record index: 812, ID: 0a92b8d5-e907-4136-9d7f-b8522ff7c25a
 - Record index: 1126, ID: 06d1bb0e-f59f-44d3-ae38-30346db6b0ed


In [18]:
import json
from pathlib import Path

# Step 1: Deduplicate
def deduplicate(dataset):
    seen = set()
    unique_examples = []
    for example in dataset:
        key = (example["Question"], example["Context"])
        if key not in seen:
            seen.add(key)
            unique_examples.append(example)
    return unique_examples

from datasets import Dataset

unique_data = deduplicate(heq_dataset)
deduplicated_dataset = Dataset.from_list(unique_data)

print(f"Original size: {len(heq_dataset)}")
print(f"Deduplicated size: {len(deduplicated_dataset)}")

# Step 2: Lowercase the field names
def lowercase_keys(example):
    return {k.lower(): v for k, v in example.items()}

lowercased_examples = [lowercase_keys(ex) for ex in unique_data]

# Step 3: Save to JSONL
output_path = "/home/nlp/achimoa/workspace/hebrew_text_retrieval/data/retrieval/queries/test.jsonl"
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
with open(output_path, "w", encoding="utf-8") as f:
    for record in lowercased_examples:
        json.dump(record, f, ensure_ascii=False)
        f.write("\n")

print(f"Saved {len(lowercased_examples)} records to {output_path}")


Original size: 1504
Deduplicated size: 1503
Saved 1503 records to /home/nlp/achimoa/workspace/hebrew_text_retrieval/data/retrieval/queries/test.jsonl


In [19]:
data_files = {
    "test": "data/retrieval/queries/test.jsonl"
}

queries_dataset = load_dataset("json", data_files=data_files)
queries_dataset

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'title', 'source', 'context', 'question', 'answers', 'is_impossible', 'wh_question', 'question_quality'],
        num_rows: 1503
    })
})